In [1]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import matplotlib.pyplot as plt

In [2]:
df = unpkl('raw.pkl')
paul = unpkl('lraw.pkl')

# STD

In [3]:
allvals = list(df.g60) + list(paul.g60)

In [4]:
g60std = np.std(allvals)
g60std

30.179949713173084

# corrections

In [5]:
df.cat.loc[df.cat=='5447'] = '5447n'

In [6]:
df = df.loc[df.cat!='5149']

In [7]:
idxs = df.loc[(df.cat=='1856')&(df.g60 < 30)].index
df = df.drop(idxs)

In [8]:
idxs = df.loc[(df.cat=='86')&(df.fname.str.contains('binder5_0'))].index
df = df.drop(idxs)

In [9]:
idxs = df.loc[(df.cat=='2498')&(df.fname.str.contains('binder5_6'))].index
df = df.drop(idxs)

In [10]:
df = df.loc[df.cat!='5151']

In [11]:
df.cat.loc[(df.cat=='4945')&(df.fname.str.contains('binder7'))] = '5554'

In [12]:
idxs = df.cat.loc[(df.cat=='2073z')&(df.manbran.str.contains('kp'))].index
df = df.drop(idxs)

In [13]:
idxs = df.loc[(df.cat.str.contains('284'))&(df.fname.str.contains('sn_'))&(df.cat!='5284')].index
df = df.drop(idxs)

In [14]:
idx = df.loc[(df.cat.str.contains('1880t'))&(df.fname.str.contains('sn_'))].index
df = df.drop(idx)

In [15]:
idx = df.loc[(df.cat.str.contains('2088'))&(df.fname.str.contains('sn_'))].index
df = df.drop(idx)

In [16]:
idxs = df.loc[(df.cat.isin(['2075ee','2075gg','2075kk']))&(df.fname.str.contains('sn_'))&(~df.fname.str.contains('fix'))].index
df = df.drop(idxs)

# Enumeration of Cases

In [17]:
df.mtrial = df.mtrial.fillna(100)

In [18]:
exact_whitelist = [
    '2293','2188','2194','2224','2063','1993','2162','2155','4231','5150','5155','5154','4949','5494j','5036'
]

In [19]:
from itertools import combinations

In [20]:
badcats = []
for cat in df.cat.unique():
    tmp = df[df.cat==cat]
    tmp.mtrial = tmp.mtrial.apply(int)
    tmp = tmp.sort_values('mtrial')
    
    # if all measurements are from same file, have same manbran, and trial indices are consecutive, then ignore
    if all([len(tmp.fname.unique())==1,len(tmp.manbran.unique())==1,is_consecutive(tmp.mtrial)]):
        continue
    elif all([len(tmp.manbran.unique())==1,tmp.g60.max()-tmp.g60.min() < g60std]):
        continue
    elif cat in exact_whitelist:
        continue
    else:
        print(cat,end='\t')
        badcats.append(cat)

In [29]:
legacy_cats = set([item.split("_")[0] for item in paul.catalog]) - set(df.cat)

In [30]:
paul['cat'] = [item.split("_")[0] for item in paul.catalog]

In [33]:
paul = paul.loc[paul.cat.isin(legacy_cats)]

In [43]:
paul.to_pickle('raw_dedupe_paul.pkl')

In [44]:
df.to_pickle('raw_dedupe.pkl')